In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Import TOPO1K, OSM and TOPO25K as GeoDataFrame

ibb = gpd.GeoDataFrame.from_file('data/clean_ibb.geojson')
osm = gpd.GeoDataFrame.from_file('data/clean_osm.geojson')
hgm = gpd.GeoDataFrame.from_file('data/clean_hgm.geojson')

In [ ]:
# Create DBpedia links from wikipedia related columns of OSM data

osm['wp'] = osm.wikipedia[osm['wikipedia'].astype(str).str.contains('en:')].str.replace(" ","_").str.replace("en:","dbr:")
osm_wp1 = osm[['fid','wp']][osm['wp'].notnull()]
osm['wp'] = ('dbr:' + osm['wikipedia_en'][osm['wikipedia_en'].notnull()].str.replace(" ","_"))
osm_wp2 = osm[['fid','wp']][osm['wp'].notnull()]
osm_wp = pd.concat([osm_wp1,osm_wp2],ignore_index=True)
dbpedia = [('idt:' + osm_wp['fid'][i] + ' owl:sameAs ' + osm_wp['wp'][i] + ' .' ) for i in range(len(osm_wp))]

In [ ]:
# Create Wikidata relations from related columns of OSM

osm['wd'] = ('<https://www.wikidata.org/wiki/' + osm['wikidata'][osm['wikidata'].notnull()] +'>')
osm_wd = osm[['fid','wd']][osm['wd'].notnull()].reset_index(drop=True)
wikidata = [('idt:' + osm_wd['fid'][i] + ' owl:sameAs ' + osm_wd['wd'][i] + ' .' ) for i in range(len(osm_wd))]

In [ ]:
# Create links of TOPO1K data from district information (Fatih)

Topo1K = [('idt:' + ibb['fid'][i] + ' schema:containedInPlace idt:Fatih .') for i in range(len(ibb))]

In [ ]:
# Create owl:SameAs links of TOPO1K data from district information (Fatih)

sameAsLinks = ['idt:Fatih owl:sameAs <http://dbpedia.org/resource/Fatih> .', 'idt:Fatih owl:sameAs <http://yago-knowledge.org/resource/Fatih> .']

In [ ]:
# Write all information to a Turtle file as links

links = dbpedia + wikidata + Topo1K + sameAsLinks

with open('data/links.ttl', 'w') as output:
        for i in range(len(links)):
            output.write('\n' + links[i] + '\n')